In [301]:
def clickButton(xPath):
    button = driver.find_element_by_xpath(xPath)
    button.click()
    
def cleanRow(row):
    raw_text = row.get_text()
    raw_text = raw_text.replace('\n','')
    raw_text = raw_text.split('\t')
    return raw_text[1],raw_text[2],raw_text[7]
    

In [351]:
driver = webdriver.Chrome(executable_path=path_to_chromeDriver)
driver.implicitly_wait(10)
driver.get(url)
clickButton('//*[@id="adult"]')
time.sleep(5)
soup = BeautifulSoup(driver.page_source,'lxml')
season_passes = soup.find("div",{"id":"season_passes"})
res = []
for row in season_passes.find_all("div",{"class":"row"})[2:]:
    res.extend([cleanRow(row)])    
res




[('Jesse Black', '4,590,214', '1'),
 ('Geoff Vevera', '2,238,487', '6'),
 ('Susan Armstrong', '2,191,944', '7'),
 ('Nancy Cohn', '1,888,811', '9'),
 ('John Gibson', '1,871,618', '11'),
 ('Michael Donnay', '1,797,001', '12'),
 ('David Archibald', '1,714,796', '13'),
 ('Coby Kluth', '1,696,801', '14'),
 ('Dow Powell', '1,553,860', '15'),
 ('Don Harring', '1,443,209', '18'),
 ('Jim Carroll', '1,391,368', '21'),
 ('Donald Lewis', '1,336,036', '24'),
 ('Jeremiah Thomas', '1,189,471', '32'),
 ('Brian Yaeger', '1,183,997', '33'),
 ('Greg Tate', '1,162,208', '36'),
 ('Amber Drysdale', '1,150,471', '37'),
 ('Scott McKinney', '1,131,004', '38'),
 ('Mike Lancaster', '1,112,180', '42'),
 ('Kent Gilkerson', '1,111,261', '43'),
 ('Carol Cunningham', '1,092,893', '44'),
 ('Jay Penrod', '1,091,571', '45'),
 ('Don Cossel', '1,082,739', '47'),
 ('Bobby Lopp', '1,049,448', '51'),
 ('Mike Morley', '1,033,843', '53'),
 ('Paul Beighle', '1,023,941', '54'),
 ('Deb Forthofer', '1,020,468', '55'),
 ('Tom Matel

In [690]:
import boto3

def pullFromS3(key):
    '''
    
    '''
    bucket = 'scrape-the-fish-bucket'
    s3 = boto3.client('s3')
    s3.download_file(bucket, key, key)


def pushToS3(path, key):
    '''
    path(str) path to file to be pushed
    key(str) filename
    
    pushes file to the s3 scrape-the-fish-bucket
    '''
    bucket = 'scrape-the-fish-bucket' # already created on S3
    s3 = boto3.resource('s3')
    s3.meta.client.upload_file(path, bucket, key)
    
    
def getS3Files(bucket='scrape-the-fish-bucket'):
    kwargs = {'Bucket':bucket}
    s3 = boto3.client('s3')
    return s3.list_objects_v2(**kwargs)
    
def getMostRecentDataFile():
    res = getS3Files()
    contents = res['Contents']
    contents.sort(key = lambda obj: obj['LastModified'], reverse = True)
    return contents[0]

    
    

In [706]:
getMostRecentDataFile()

{'Key': 'data20210317.csv',
 'LastModified': datetime.datetime(2021, 3, 17, 22, 35, 2, tzinfo=tzutc()),
 'ETag': '"4834260a1442db736e66cad4c3bd9a58"',
 'Size': 49448,
 'StorageClass': 'STANDARD'}

In [698]:
df = pullFromS3(res['Key'])

In [702]:
res['Key']

'data20210317.csv'

In [518]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from datetime import datetime, timedelta, date

url = 'https://skiwhitefish.com/vertical-tracker/'
path_to_chromeDriver = '/Users/drahcir1/Documents/chromedriver'

In [716]:
class LeaderBoard:
    
    def __init__(self, driver, output=[]):
        
        self.driver = driver
        if self.driver.current_url != 'https://skiwhitefish.com/vertical-tracker/':
            self.driver.get('https://skiwhitefish.com/vertical-tracker/')
        self.output = output
    
    def __clickButton(self,xPath):
        button = self.driver.find_element_by_xpath(xPath)
        button.click()
        
    def __cleanRow(self,row):
        raw_text = row.get_text()
        raw_text = raw_text.replace('\n','')
        raw_text = raw_text.split('\t')
        return {"name":raw_text[1],
                "vertical":int(raw_text[2].replace(',','')),
                "ranking":int(raw_text[7].replace(',',''))}
    
    def getS3Files(self, bucket='scrape-the-fish-bucket'):
        kwargs = {'Bucket':bucket}
        s3 = boto3.client('s3')
        return s3.list_objects_v2(**kwargs)

    def getMostRecentDataFile(self):
        res = getS3Files()
        contents = res['Contents']
        contents.sort(key = lambda obj: obj['LastModified'], reverse = True)
        return contents[0]


        
    def getLeaderBoardDump(self,category,categoryButton):
        self.__clickButton(categoryButton)
        self.today = date.today()
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source,'lxml')
        season_passes = soup.find("div",{"id":"season_passes"})
        for row in season_passes.find_all("div",{"class":"row"})[2:]:
            cleanData = self.__cleanRow(row)
            self.output.extend([{'date':self.today,
                                 "category":category, 
                                 'name':cleanData['name'],
                                 'vertical':cleanData['vertical'],
                                 "ranking":cleanData['ranking']
                                }])    
    
    def getAllBoards(self):
        self.getLeaderBoardDump("supersenior",'//*[@id="super_senior"]')
        self.getLeaderBoardDump("senior",'//*[@id="senior"]')
        self.getLeaderBoardDump("adult",'//*[@id="adult"]')
        self.getLeaderBoardDump("teen",'//*[@id="teen"]')
        self.getLeaderBoardDump("junior",'//*[@id="junior"]')
        self.getLeaderBoardDump("child",'//*[@id="child"]')
        self.getLeaderBoardDump("college",'//*[@id="college"]')
        return self.output
        
        
        
    
driver = webdriver.Chrome(executable_path=path_to_chromeDriver)
driver.implicitly_wait(5)
Board = LeaderBoard(driver)
res = Board.getAllBoards()
res.sort(key = lambda skier: skier['ranking'])
driver.close()

df = pd.DataFrame(res)
df.index.name = 'index'


yesterday = (datetime.now() - timedelta(1)).strftime('%Y%m%d')
today = datetime.today().strftime('%Y%m%d')

lastFile = getMostRecentDataFile()
lastFile = lastFile['Key']
pullFromS3(lastFile)
df_yesterday = pd.read_csv(lastFile, index_col = 'index')


df = df.append(df_yesterday)

df_filename = 'data{}.csv'.format(today)
backup_filename = 'backup{}.csv'.format(today)

df.to_csv(df_filename)
df.to_csv(backup_filename)

pushToS3(df_filename,df_filename)
pushToS3(backup_filename,df_filename)

In [602]:
pushToS3(df_filename,"{}.csv".format(df_filename))
pushToS3(backup_filename,"{}.csv".format(backup_filename))


In [659]:
df.head(60)

,date,category,name,vertical,ranking
index,,,,,
0,2021-03-23,adult,Jesse Black,5197411,1
1,2021-03-23,supersenior,Jim Petersen,3194600,2
2,2021-03-23,senior,Laurence Kenik,3063373,3
3,2021-03-23,supersenior,Fred Frost,2673981,4
4,2021-03-23,adult,Susan Armstrong,2537969,5
5,2021-03-23,adult,Geoff Vevera,2513349,6
6,2021-03-23,senior,Randy Helgath,2436932,7
7,2021-03-23,senior,Duane Bauch,2305582,8
8,2021-03-23,adult,Michael Donnay,2266949,9


['/Users/drahcir1/Documents/scrapeTheFish']